## Extract Spectral Libraries from Imagery
This code uses a reference polygon dataset of plant species to extract spectra from imagery. 

* Can extract spectra from single image or multiple images in a directory. If using a directory, image spectra can be saved separately or as a single file. 
* Code can be run without metadata, metadata provided as .csv, and metadata from shapefile

Susan Meerdink  
April 2019

In [1]:
# Import Modules
import glob
import numpy as np
import rasterio.plot
from rasterio import features
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import fiona
from descartes import PolygonPatch
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import pyproj
from functools import partial
import random
import gc
from os.path import basename
import os
import utils

TO DO:
it is saving the output file into the same folder as input and not renaming
the header is not outputting the bands names (0,0)

In [2]:
# # Processing Single file without metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\input_shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# utils.create_spec_lib(polyLocation, imageLocation, outLocation, 0, 0)

In [3]:
# WHISPERS Conference 2019 - 20190903 Init
polyLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Validation_polygons\\SB_092014_VALID'
imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\20190903\\f140829t01p00r09_corr_v1_urb_10000_SMA_20190903T14H11M06S_init_sn_rot_geo'
outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r09_20190903_init_fractions.csv'
create_spec_lib(polyLocation, imageLocation, outLocation, 0)

imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\20190903\\f140829t01p00r10_corr_v1_urb_10000_SMA_20190903T18H12M43S_init_sn_rot_geo'
outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\20190903\\f140829t01p00r10_20190903_init_fractions.csv'
create_spec_lib(polyLocation, imageLocation, outLocation, 0)

NameError: name 'create_spec_lib' is not defined

In [ ]:
# # WHISPERS Conference 2019 - 6 Target with ACE proportions
# polyLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Validation_polygons\\SB_092014_VALID'
# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r09_ACE_Alpha005_Conf_noopt_rot_geo'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r09_ACE_Alpha005_Conf_noopt_rot_geo_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r10_ACE_Alpha005_Conf_noopt_rot_geo'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r10_ACE_Alpha005_Conf_noopt_rot_geo_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\MESMA\\18m_mos_nan_SA'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\MESMA\\18m_mos_nan_SA_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

In [ ]:
# # WHISPERS Conference 2019 - 6 Target
# polyLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Validation_polygons\\SB_092014_VALID'
# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r09_corr_v1_urb_10000_SMA_20190514T15H45M47S_shadenorm_rot_geo'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r09_corr_v1_urb_10000_SMA_20190514T15H45M47S_shadenorm_rot_geo_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r10_corr_v1_urb_10000_SMA_20190515T00H47M22S_shadenorm_rot_geo'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r10_corr_v1_urb_10000_SMA_20190515T00H47M22S_shadenorm_rot_geo_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)


In [ ]:
# # Processing single file with shapefile metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

In [ ]:
# # Processing single file with .csv metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# metaLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\meta.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0, [metaLocation, '', ''])

In [ ]:
# Processing multiple files without metadata separately

In [ ]:
# Processing multiple files with shapefile metadata separately

In [ ]:
# Processing multiple files with .csv metadata separately

In [ ]:
# Processing multiple files without metadata together

In [ ]:
# Processing multiple files with shapefile metadata together

In [ ]:
# Processing multiple files with .csv metadata together